In [1]:
import os

# find directories
thedir='.'
directories=[ name for name in os.listdir(thedir) if os.path.isdir(os.path.join(thedir, name)) ]

# find html files
menu=[]
for directory in directories:
    htmls=[html for html in os.listdir(directory) if html[-5:]=='.html']
    htmls.sort()
    if htmls!=[]:
        menu+=[[directory,htmls]]

# get menu data from htmls
from bs4 import BeautifulSoup as soup

def getItemData(directory,file):
    with open(f'{directory}/{file}') as fp:
        html = soup(fp, 'html.parser')
    return [html.article.div.text,f'/lorentz/{directory}/{file}',f'{file.split(".")[0]} — {html.h2.text}']

details=[]
for directory in menu:
    details+=[getItemData(directory[0],file) for file in directory[1]]

# group details
group={}
for item in details:
    if item[0] in group:
        group[item[0]]+=[item[1:]]
    else:
        group[item[0]]=[item[1:]]

# write new nav.js file and backup
from datetime import datetime

with open('nav.js','r') as nav:
    navlines=nav.readlines()

with open(f'bak/{int(datetime.timestamp(datetime.now()))}.js','w') as backup:
    backup.writelines(navlines)

navlines[0]=f"var menuItems = {str(group)}\n"

with open('nav.js','w') as newNav:
    newNav.writelines(navlines)